# Исследование надежности заемщиков.

**Заказчик**

Кредитный отдел банка.

**Цель исследования** — проверить гипотезы и отвеить на вопрос:

* Есть ли зависимость между количеством детей и возвратом кредита в срок.
* Есть ли зависимость между семейным положением и возвратом кредита в срок.
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок.
* Как разные цели кредита влияют на его возврат в срок?

**Входные данные от банка**

Статистика о платёжеспособности клиентов.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas
df = pd.read_csv('data.csv') # чтение файла с данными и сохранение в df


Выведем на экран первые десять строк таблицы:

In [2]:
display(df.head(10)) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


Итак, в таблице 12 столбцов с разными типами данных.

Описание данных:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Изучая данные в таблице, мы обнаруживаем следующие нарущения:
1. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.
2. В данных встречаются аномалии. Отрицательное количество дней трудового стажа в столбце days_employed.
3. Неооходимо привезти к общему стандарту написания категориальных данных  столбец education.
4. А также необходимо проверить на наличие явных и неявных дубликатов.

Перед тем как решать поставленные вопросы, необходимо устранить проблемы в данных.

### Шаг 2.1 Проверка данных на аномалии и исправления.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed.
Рассотрим какие данные хранятся в нашем столбце, для этого воспользуемся методом **describe()**


In [4]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Рассмотрим полученные данные: всего строк 21525, минимальное значение -18388, а максимальное значение 401755. Отрицательным стаж никак не может быть. Возможно это человеческий фактор. 

Заменим отрицательные значения на абсолютные, для этого используем loc c условием:

In [5]:
df.loc[df['days_employed'] < 0, 'days_employed'] = abs(df['days_employed'])

Проверим, что что у нас получилось, для этого вызовем еще раз метод describe()

In [6]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

В результате мы заменили все отрицательные данные на положительные. При этом количество данных у нас не изменилось, но теперь минимальное значение 24, а максимальное не изменилось и осталось 401755.


### Шаг 2.2 Заполнение пропусков

В двух столбцах есть пропущенные значения, а именно в столбцах days_employed (общий трудовой стаж в днях) и total_income(ежемесячный доход). В них хранятся количественные переменные. Пропуски в таких переменных заполняют характерными значениями. Для этого обычно используют среднее арифметическое или медиану.

Для начала рассчитаем какую долю составляют пропуски в каждом из столбцов. 
В связи с тем, что общее количество строк с данными одинаковое в обоих случаях, расчеты мы будем производить для одного из столбцов.

In [7]:
(df['days_employed'].isna().sum()/df['days_employed'].count())*100 #доля для days_employed

11.234561521368406

In [8]:
(df['days_employed'].isna().sum()/len(df['days_employed']))*100

10.099883855981417

Таким образом получаем, что доля пропущенных данный в обоих столбцах составила **10%**.

Каковы же могут быть причины возникновения пропущенных данных. Предположим несколько вариантов:
* эти поля не были обязательными для заполнения и клиент не захотел указывать эти данные
* возможно клиент никогда не работал или уже на пенсии

Для заполнения пропусков расчитаем медиану. Этот параметр (в отличие от среднего значения) устойчив к «выбросам».

In [9]:
df['days_employed'].median() #расчитаем медиану для days_employed

2194.220566878695

In [10]:
df['total_income'].median() #расчитаем медиану для total_income

145017.93753253992

Заполним пропуски полученными данными:

In [11]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) #заполняем пропуски медианным значением
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [12]:
df.isna().sum() # подсчёт пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце total_income на целочисленный. Для этого воспользуемся методом astype().

In [13]:
df['total_income'] = df['total_income'].astype(int) #меняем float на int методом astype()

Проверим, что у нас получилось. Для этого вызовем info()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.5+ MB


Таким образом мы заменили вещественный тип данных на целочисленный.

### Шаг 2.4. Удаление дубликатов.

Сначала найдем явные дубликаты и посчитаем сколько таких строк.

In [15]:
print(df.duplicated().sum()) 

54


Теперь с помощью специального метода удалим их:

In [16]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Еще раз посчитаем явные дубликаты, чтобы убедиться, что мы их удалили

In [17]:
print(df.duplicated().sum()) 

0


Теперь осталось избавиться от неявных дубликатов. Например, одна и та же информация может быть записана немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Для начала проверим на уникальность следующие столбцы: `education`, `family_status`, `income_type`.


In [18]:
# Просмотр уникальных названий уровней образования
print(sorted(df['education'].unique()))

['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']


In [19]:
# Просмотр уникальных семейных статусов
print(sorted(df['family_status'].unique()))

['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']


In [20]:
# Просмотр уникальных типов занятости
print(sorted(df['income_type'].unique()))

['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']


Таким образом мы видим, что дубликаты есть только в столбце с уровнем образования. Одна и та же информация записана по-разному ( то строчными, то заглавными буквами). Для того, чтобы это исправить приведем все строки в нижний регистр методом str.lower()

In [21]:
df['education'] = df['education'].str.lower() #преобразуем все строки колонки education в нижний регистр

Проверим как теперь выглядят строки в столбце:

In [22]:
print(sorted(df['education'].unique())) ## Просмотр уникальных названий

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']


Теперь с помощью специального метода удалим их:

In [23]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Еще раз посчитаем явные дубликаты, чтобы убедиться, что мы их удалили

In [24]:
print(df.duplicated().sum()) 

0


**Выводы**

При изучении полученных данных были обнаружены три проблемы:

- аномалии в данных,
- пропущенные значения,
- дубликаты — явные и неявные.

Без дубликатов и аномалий исследование станет более точным.

Пропуски мы заменили на `медианные значения`. 

Теперь можно перейти к проверке гипотез. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для того чтобы ответить на поставленные вопросы, нам необходимо наши данные сначала преобразовать.

Для начала сделаем два "словаря" датафрейма:
* семейное положение
* уровень образования

In [25]:
df_education=df[['education_id', 'education']] #созданим словарь семейного положения
display(df_education.head(10))

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


В «словаре» мы заметили большое количество дубликатов. Их нужно удалить. 

In [26]:
df_education = df_education.drop_duplicates().reset_index(drop=True) #Применим методы: drop_duplicates() и reset_index().
display(df_education) #проверим результат

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Аналогично создадим второй "словарь" семейное положение и удалим в нем дубликаты:

In [27]:
df_family = df[['family_status_id', 'family_status']] #создаем новый датафрейм
df_family = df_family.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
display(df_family)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Теперь удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id.

In [28]:
df=df.drop(['education', 'family_status'], axis=1) #удалим колонки с помощью метода drop()
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Для этого создадим функцию total_income_category, которая будет делить уровень дохода по категориям указанным выше.

In [29]:
def total_income_category(total_income):
    if total_income<=30000:
        return 'E'
    if total_income<=50000:
        return 'D'
    if total_income<=200000:
        return 'C'
    if total_income<=1000000:
        return 'B'
    if total_income>1000001:
        return 'A'

Создаем отдельный столбец total_income_category, в ячейки которого будут записаны категории, которые вернет созданная функция.

In [30]:
#Для этого нужен метод apply(): он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента.
df['total_income_category'] = df['total_income'].apply(total_income_category)
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Cформируем новый столбец purpose_category, в который войдут следующие категории:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение сявадьбы`
* `получение образовани`

Для этого создадим новую функцию `purpose_category` и используем метод `apply()`

Перед тем как создавать функцию нам необходимо изучить какие цели получения кредита есть в таблице с данными. 

In [31]:
print(df['purpose'].unique()) #проверим уникальные значения в столбце purpose

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Мы видим, что цели можно разбить на четыре категории, причем такие цели как "покупка жилья" и "операции с недвижимостью" можно объединить в одну категорию "операции с недвижимостью".

In [32]:
# создаем функцию, которая будет на вход принимать значения purpose, а на выходе создавать категорию
def purpose_category(purpose): 
    if 'жил'  in purpose or 'недвиж'  in purpose:
        return 'операции с недвижимостью'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    

In [33]:
#используем метод apply, он берёт значения столбца purpose и применяет к ним функцию purpose_category из своего аргумента
df['purpose_category']=df['purpose'].apply(purpose_category)
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [34]:
print(df['purpose_category'].unique()) #проверим какие категории у нас получились и все ли данные распределились

['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


В итоге получили дополнительный столбец `purpose_category` с разбивкой на четыре категории:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение сявадьбы`
* `получение образовани`

### Ответы на вопросы.

##### Вопрос 1:

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того чтобы ответить на этот вопрос создадим сводную таблицу с помощью метода pivot_table()

In [35]:
#сводная таблица методом pivot_table, где строки количество детей, а столбцы - была ли задолжность
pivot_children = df.pivot_table(index='children', columns='debt', values='purpose_category', aggfunc='count')
pivot_children

debt,0,1
children,,
-1,46.0,1.0
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


Благодаря этой сводной таблице, мы видим еще одну аномалию - отрицательное число и 20 детей. Скорей всего это человеческий фактор или сбой при выгрузке данных. Исправим этот недочет.

In [36]:
#доля по отрицательным значениям
len(df[df['children']==-1])/len(df['children'])*100

0.21907336627202387

In [37]:
#доля по значению 20 детей
len(df[df['children']==20])/len(df['children'])*100

0.3542462943973152

Получаем по каждой аномалии доли меньше одного процента (которые не сможет оказать влияние на расчет) и соответственно можем смело исправить эти ошибки.

In [38]:
# заменим отрицательное значение на положительное и 20 детей исправим на 2 ребенка с помощью loc и повторно сформируем сводную таблицу
df.loc[df['children'] < 0, 'children'] = abs(df['children']) #найдем и заменим отрицательное значение
df.loc[df['children']==20, 'children'] = 2
pivot_children = df.pivot_table(index='children', columns='debt', values='purpose_category', aggfunc='count')
pivot_children

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [39]:
# Посчитаем уникальные значения столбца с данными пола методом value_counts()
df['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

Мы обнаружили еще одну аномалию - это единичное значение XNA. Тк этот столбец не используется в расчетах и все таки это единичный случай, который ни на что не может повлиять, поэтому мы можем оставить эту строку без изменений.

Изучая полученную таблицу, мы видим зависимость задолжности от количества детей, то есть с увеличением детей уменьшается количество фактов задолжности. Но также мы видим уменьшение количества кредитов без задолжностей. Поэтому однозначно ответить на поставленный вопрос мы пока не можем. 



In [40]:
#соберем новый сводник, где сразу сделаем аггрегацию по среднему, сумме и кол-ву  (aggfunc=['count', 'sum', 'mean']),
# и в результате получим долю:
df.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])


,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


В результате получаем следующие данные:
 * count- общее количество строк по каждой категории детей
 * sum - количество просрочек (благодаря тому что у нас есть два значение 0 и 1, поэтому мы сложили сколько всего 1 по категории)
 * mean - таким образом мы получили долю ( отношение количества единиц поделенное на общее количество строк по каждой категории)
    

##### Вывод 1:

Данные показвают, что:
* когда у клиента детей нет вообще, то процент задолжностей явно ниже, чем у клиентов с детьми
* когда дети есть, то с увеличением количества детей процент задолжностей незначительно но тоже увеличивается
* с увеличением количества детей, количество выданных кредитов значительно сокращается.

Значит, однозначно можно сказать, что зависимость между наличием/отсутствием детей и возвратов кредита в срок есть, а вот уже непосредственно от количества детей зависимость уже не такая явная.  

##### Вопрос 2:

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для того чтобы ответить на этот вопрос создадим сводную таблицу с помощью метода pivot_table()

In [41]:
#сводная таблица методом pivot_table, где строки семейное положение, а столбцы - была ли задолжность
pivot_family_status = df.pivot_table(index='family_status_id', columns='debt', values='purpose_category', aggfunc='count')
pivot_family_status

debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


Из этой сводной таблицы не понятно где какой статус семейного положения. Для этого мы выгрузим ранее созданный словарь df_family

In [42]:
display(df_family)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Но так смотреть не удобно, поэтому соединим две таблицы в одну

In [43]:
#с помощью метода merge() соединим словарь и сводную таблицу
df_new = df_family.merge(pivot_family_status, on='family_status_id', how='left')
df_new

,family_status_id,family_status,0,1
0,0,женат / замужем,11408,931
1,1,гражданский брак,3763,388
2,2,вдовец / вдова,896,63
3,3,в разводе,1110,85
4,4,Не женат / не замужем,2536,274


In [44]:
#добавим новый столбец с соотношением задолжности к общему числу по каждому статусу семейного положения
df_new['conv']=(df_new[1]/(df_new[1]+df_new[0]))*100 
df_new=df_new.sort_values(by='conv', ascending=False) #отсортируем по убыванию новый столбец 'conv'
df_new

,family_status_id,family_status,0,1,conv
4,4,Не женат / не замужем,2536,274,9.750890
1,1,гражданский брак,3763,388,9.347145
0,0,женат / замужем,11408,931,7.545182
3,3,в разводе,1110,85,7.112971
2,2,вдовец / вдова,896,63,6.569343


##### Вывод 2:

Если сравнить полученные процентные соотношения, то мы можем сделать следующий вывод:
* самый надежный клиент со статусом `вдовец / вдова`
* самый не надежный со статусом `Не женат / не замужем`
    
Соответственно мы можем сделать вывод, что существует зависимость между семейным положением и возвратом кредита в срок.

##### Вопрос 3:

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Ранее мы создавали категории по уровням дохода и создали следующую градацию:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Эти категории были записаны в новый столбец total_income_category.

Используем эти данные для поиска зависимости возврата кредита в срок. для этого создадим сводную таблицу, где строки - уровни дохода, а столбцы - была ли задолжность 

In [45]:
# создадим сводную таблицу с помощью pivot_table()
pivot_total_incom = df.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])
pivot_total_incom

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5041,356,0.070621
C,16016,1360,0.084915
D,350,21,0.060000
E,22,2,0.090909


##### Вывод 3:

Из полученных данных мы видим, что больше всего кредитов было выдано клиентам с уровнем дохода 50001–200000 и меньше всего выдавались кредиты самым бедным и самым богатым.

При уменьшении уровня дохода мы не наблюдаем равномерного изменения процентного соотношения, соответственно мы может сделать вывод, что зависимости нет между уровнем дохода и возвратом кредита в срок.

##### Вопрос 4:

#### Как разные цели кредита влияют на его возврат в срок?

Ранее мы разбивали цели кредита на четыре типа:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение сявадьбы`
* `получение образовани`

Рассмотрим как влияют цели на возврат кредита. Для этого создадим сводную таблицу, где в строках будут цели, а в столбцах возврат кредита.

In [46]:
# создадим сводную таблицу с помощью pivot_table()
pivot_purpose_category = df.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean'])
pivot_purpose_category

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10811,782,0.072334
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


##### Вывод 4:

Если сравнивать процентные соотношения, то мы можем разбить наши цели кредита на две группы:
   1. операции с автомобилем и получение образования (около 9%)
   2. операции с недвижимостью и проведение свадьбы (около 7%)
    
При чем вторая группа подходит более ответственно к возврату кредита, чем первая группа.
Самая востребованная и как показало процентное соотношение самая надежная категория - это операции с недвижимостью.

## Общий вывод:

Мы рассмотрели четыре вопроса и пришли к следующим результатам:
1. мы может с уверенностью сделать вывод, что существует зависимость между семейным положением и возвратом кредита в срок.
При чем самые надежные клиенты со статусом вдовец / вдова.

2. зависимость между количеством детей и возвратом кредита в срок частично подтверждается, так данные показали следующие закономерности:
* когда у клиента детей нет вообще, то процент задолжностей явно ниже, чем у клиентов с детьми
* когда дети есть, то с увеличением количества детей процент задолжностей незначительно, но все же увеличивается.

То есть зависимость четко прослеживается между наличием/отсутствием детей и возвратом кредита, а не с увеличением количества детей.

3. При уменьшении уровня дохода мы не наблюдаем равномерного изменения процентного соотношения, соответственно мы может сделать вывод, что зависимости нет между уровнем дохода и возвратом кредита в срок.
4. В результате исследования мы выделили самую надежную категорию клиентов, которые указали цель- операции с недвижимостью.